In [11]:
# libraries
import requests
import random

# !pip install ipytest
import ipytest
ipytest.autoconfig()

In [21]:
%%run_pytest[clean]

# User tests

base_url = 'http://localhost:8000'

def get_token_and_user_id(username, password):
    response = requests.post(base_url + '/login/', {
        "username": username,
        "password": password
    })
    return response.json()

def create_new_user(username, email, password):
    response = requests.post(base_url + '/users/', {
        "username": username,
        "email": email,
        "password": password
    })
    return response.json()

def get_user_by_id(user_id):
    response = requests.get(base_url + '/users/' + user_id + '/')
    return response.json()

# POST /login/
def test_if_response_for_login_has_id_and_token_keys():
    json_response = get_token_and_user_id("ted", "Ro0+1234")
    assert "token" in json_response.keys()
    assert "id" in json_response.keys()

# POST /users/
def test_if_response_for_create_user_has_id_username_and_password_keys():
    username = "".join([random.choice('abcdefghijklmnopqrstuvywxz') for _ in range(6)]) + "_test"
    email = username + "@gmail.com"
    password = "Ro0+1234"
    json_response = create_new_user(username, email, password)
    assert "id" in json_response.keys()
    assert "username" in json_response.keys()
    assert "email" in json_response.keys()
    
# GET /users/<id>/
def test_if_response_for_get_user_match_user_id():
    username = "".join([random.choice('abcdefghijklmnopqrstuvywxz') for _ in range(6)]) + "_test"
    email = username + "@gmail.com"
    password = "Ro0+1234"
    new_user = create_new_user(username, email, password)
    user_id = new_user['id']
    json_response = get_user_by_id(str(user_id))
    assert json_response['id'] == user_id

...                                                                                                              [100%]
3 passed in 9.61s


In [94]:
%%run_pytest[clean]

base_url = 'http://localhost:8000'

# People tests

# GET /people/
def test_if_people_list_return_count_results_next_previous():
    user = get_token_and_user_id("ted", "Ro0+1234")
    token = user['token']
    response_json = requests.get(base_url + '/api/v1/people/', headers={
        "Authorization": "Token {}".format(token),
        "Content-Type": "application/json",
    }).json()
    assert "results" in response_json.keys()
    assert "count" in response_json.keys()    
    
# POST /people/
def test_if_create_people_return_201():
    user = get_token_and_user_id("ted", "Ro0+1234")
    token = user['token']    

    name = "".join([random.choice('abcdefghijklmnopqrstuvywxz') for _ in range(6)]) + " test"

    response = requests.post(base_url + '/api/v1/people/', headers={
        "Authorization": "Token {}".format(token),
        "Content-Type": "application/json",
    }, json={
        "name": name
    })
    response_json = response.json()
    assert response.status_code == 201
    assert "id" in response_json.keys()
    assert "name" in response_json.keys()
    assert response_json["name"] == name.upper()
    
# GET /api/v1/people/<id>/
def test_if_find_people_by_id_return_correct_data():
    user = get_token_and_user_id("ted", "Ro0+1234")
    token = user['token']    

    name = "".join([random.choice('abcdefghijklmnopqrstuvywxz') for _ in range(6)]) + " test"

    people = requests.post(base_url + '/api/v1/people/', headers={
        "Authorization": "Token {}".format(token),
        "Content-Type": "application/json",
    }, json={
        "name": name
    }).json()
    
    people_id = people['id']
    
    response_json = requests.get(base_url + '/api/v1/people/' + str(people_id) + '/', headers={
        "Authorization": "Token {}".format(token),
        "Content-Type": "application/json",        
    }).json()
    
    assert response_json['name'] == name.upper()
    assert response_json['id'] == people_id

# PUT /people/<id>/
def test_if_people_are_correctly_updated():
    user = get_token_and_user_id("ted", "Ro0+1234")
    token = user['token']    

    name = "".join([random.choice('abcdefghijklmnopqrstuvywxz') for _ in range(6)]) + " test"

    people = requests.post(base_url + '/api/v1/people/', headers={
        "Authorization": "Token {}".format(token),
        "Content-Type": "application/json",
    }, json={
        "name": name
    }).json()
    
    people_id = people['id']
    
    other_name = "".join([random.choice('abcdefghijklmnopqrstuvywxz') for _ in range(6)]) + " test"
    
    response_json = requests.put(base_url + '/api/v1/people/' + str(people_id) + '/', headers={
        "Authorization": "Token {}".format(token),
        "Content-Type": "application/json",        
    }, json={
        "name": other_name
    }).json()
    
    assert response_json['name'] == other_name.upper()
    assert response_json['name'] != name.upper()

# PATCH /people/<id>/
def test_if_people_are_correctly_updated():
    user = get_token_and_user_id("ted", "Ro0+1234")
    token = user['token']    

    name = "".join([random.choice('abcdefghijklmnopqrstuvywxz') for _ in range(6)]) + " test"

    people = requests.post(base_url + '/api/v1/people/', headers={
        "Authorization": "Token {}".format(token),
        "Content-Type": "application/json",
    }, json={
        "name": name
    }).json()
    
    people_id = people['id']
    
    mother_name = "".join([random.choice('abcdefghijklmnopqrstuvywxz') for _ in range(6)]) + " test"
    
    response_json = requests.patch(base_url + '/api/v1/people/' + str(people_id) + '/', headers={
        "Authorization": "Token {}".format(token),
        "Content-Type": "application/json",        
    }, json={
        "mother_name": mother_name
    }).json()
    
    assert response_json['mother_name'] == mother_name.upper()

# GET /people/?search=<name>
def test_if_people_are_correctly_searched():
    user = get_token_and_user_id("ted", "Ro0+1234")
    token = user['token']    

    name1 = "".join([random.choice('abcdefghijklmnopqrstuvywxz') for _ in range(6)]) + " findme"
    name2 = "".join([random.choice('abcdefghijklmnopqrstuvywxz') for _ in range(6)]) + " findme"
    name3 = "".join([random.choice('abcdefghijklmnopqrstuvywxz') for _ in range(6)]) + " findme"

    requests.post(base_url + '/api/v1/people/', headers={
        "Authorization": "Token {}".format(token),
        "Content-Type": "application/json",
    }, json={
        "name": name1
    })
    
    requests.post(base_url + '/api/v1/people/', headers={
        "Authorization": "Token {}".format(token),
        "Content-Type": "application/json",
    }, json={
        "name": name2
    })
    
    requests.post(base_url + '/api/v1/people/', headers={
        "Authorization": "Token {}".format(token),
        "Content-Type": "application/json",
    }, json={
        "name": name3
    })
        
    response_json = requests.get(base_url + '/api/v1/people?search=findme', headers={
        "Authorization": "Token {}".format(token),
        "Content-Type": "application/json",        
    }).json()
    
    assert len(response_json['results']) == 3

# DELETE /people/<id>/
def test_if_delete_all_created_data_done():
    user = get_token_and_user_id("ted", "Ro0+1234")
    token = user['token']

    search_created_data = requests.get(base_url + '/api/v1/people?search=findme', headers={
        "Authorization": "Token {}".format(token),
        "Content-Type": "application/json",        
    }).json()['results']

    test_created_data = requests.get(base_url + '/api/v1/people?search=test', headers={
        "Authorization": "Token {}".format(token),
        "Content-Type": "application/json",        
    }).json()['results']

    ids_to_delete = list(map(lambda x: x['id'], test_created_data + search_created_data)) 
    print(ids_to_delete)
    while len(ids_to_delete) > 0:
        for people_id in ids_to_delete:
            requests.delete(base_url + '/api/v1/people/' + str(people_id) + '/', headers={
                "Authorization": "Token {}".format(token),
                "Content-Type": "application/json",        
            })
        search_created_data = requests.get(base_url + '/api/v1/people?search=findme', headers={
            "Authorization": "Token {}".format(token),
            "Content-Type": "application/json",        
        }).json()['results']

        test_created_data = requests.get(base_url + '/api/v1/people?search=test', headers={
            "Authorization": "Token {}".format(token),
            "Content-Type": "application/json",        
        }).json()['results']
        ids_to_delete = list(map(lambda x: x['id'], test_created_data + search_created_data)) 
    
    search_data_after_delete = requests.get(base_url + '/api/v1/people?search=findme', headers={
        "Authorization": "Token {}".format(token),
        "Content-Type": "application/json",        
    }).json()['results']
    
    test_data_after_delete = requests.get(base_url + '/api/v1/people?search=test', headers={
        "Authorization": "Token {}".format(token),
        "Content-Type": "application/json",        
    }).json()['results']   
    
    assert len(test_data_after_delete) == 0
    assert len(search_data_after_delete) == 0


......                                                                                                           [100%]
6 passed in 61.33s (0:01:01)
